In [ ]:
import pandas as pd
from dataset import read_ner_file
import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset
import numpy as np 
import os
import matplotlib.pyplot as plt 
from transformers import AdamW
from tqdm import tqdm 
import torcheval 

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = torch.device("cuda")

LABEL_2_ID = {'B-PATIENT_ID': 0, 
    'I-PATIENT_ID': 1, 
    'B-NAME': 2, 
    'I-NAME': 3, 
    'B-AGE': 4, 
    'I-AGE': 5, 
    'B-GENDER': 6, 
    'I-GENDER': 7, 
    'B-JOB': 8, 
    'I-JOB': 9, 
    'B-LOCATION': 10, 
    'I-LOCATION': 11, 
    'B-ORGANIZATION': 12, 
    'I-ORGANIZATION': 13, 
    'B-SYMPTOM_AND_DISEASE': 14, 
    'I-SYMPTOM_AND_DISEASE': 15, 
    'B-TRANSPORTATION': 16, 
    'I-TRANSPORTATION': 17, 
    'B-DATE': 18, 
    'I-DATE': 19, 
    'O': 20
}

ID_2_LABEL = {0: 'B-PATIENT_ID', 
    1: 'I-PATIENT_ID', 
    2: 'B-NAME', 
    3: 'I-NAME', 
    4: 'B-AGE', 
    5: 'I-AGE', 
    6: 'B-GENDER', 
    7: 'I-GENDER', 
    8: 'B-JOB', 
    9: 'I-JOB', 
    10: 'B-LOCATION', 
    11: 'I-LOCATION', 
    12: 'B-ORGANIZATION', 
    13: 'I-ORGANIZATION', 
    14: 'B-SYMPTOM_AND_DISEASE', 
    15: 'I-SYMPTOM_AND_DISEASE', 
    16: 'B-TRANSPORTATION', 
    17: 'I-TRANSPORTATION', 
    18: 'B-DATE', 
    19: 'I-DATE', 
    20: 'O'
}